In [1]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import numpy as np
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials, space_eval
import matplotlib as plt
import time
import pickle

In [2]:
def run_annealing(prob, decay_ob, init_temp, ints, max_at, max_it):
    if decay_ob[0] == "geom":
        schedule = mlrose.GeomDecay(init_temp = init_temp,decay=decay_ob[1])
    elif decay_ob[0] == "exp":
        schedule = mlrose.ExpDecay(init_temp = init_temp, exp_const=decay_ob[1])
    start = time.time()
    _, best_fitness, fitness_curve = mlrose.simulated_annealing(prob, 
                                                             schedule = schedule, 
                                                             max_attempts = max_at, 
                                                             max_iters = max_it, 
                                                             init_state = ints, 
                                                             curve = True)
    end = time.time()
    sa_time = end - start
    return best_fitness, fitness_curve, sa_time

def run_genetic(prob, psize, mut, max_at, max_it):
    start = time.time()
    _, best_fitness, fitness_curve = mlrose.genetic_alg(prob,
                                                        pop_size = psize,
                                                        mutation_prob = mut,
                                                        max_attempts = max_at,
                                                        max_iters = max_it,
                                                        curve = True)
    end = time.time()
    ga_time = end - start
    return best_fitness, fitness_curve, ga_time

def run_mimic(prob, psize, kpct, max_at, max_it):
    start = time.time()
    _, best_fitness, fitness_curve = mlrose.mimic(prob,
                                                        pop_size = psize,
                                                        keep_pct = kpct,
                                                        max_attempts = max_at,
                                                        max_iters = max_it,
                                                        fast_mimic = True,
                                                        curve = True)
    end = time.time()
    mm_time = end - start
    return best_fitness, fitness_curve, mm_time

def run_rhc(prob,rets,ints,max_at,max_it):
    start = time.time()
    _, best_fitness, fitness_curve = mlrose.random_hill_climb(prob,
                                                              max_attempts = max_at,
                                                              max_iters = max_it,
                                                              init_state = ints,
                                                              restarts = rets,
                                                              curve = True)
    end = time.time()
    mm_time = end - start
    return best_fitness, fitness_curve, mm_time

In [3]:
def flipFlop(params):
    fitness = mlrose.FlipFlop()
    problem = mlrose.DiscreteOpt(length = params['prob_size'], 
                                 fitness_fn = fitness, 
                                 maximize = True, 
                                 max_val = 2)
    ints = np.random.randint(2, size = params['prob_size'])  
    if params['alg'] == "annealing":
        bf,fc,time_diff= run_annealing(problem,
                                       params['decay_ob'],
                                       params['init_temp'],
                                       ints,
                                       params['max_attempts'],
                                       params['max_iterations']
                                      )
    elif params['alg'] == "rhc":
        bf,fc,time_diff= run_rhc(problem,
                                 params['restart'],
                                 ints,
                                 params['max_attempts'],
                                 params['max_iterations']
                                )
    elif params['alg'] == "genetic":
        bf,fc,time_diff = run_genetic(problem,
                                      params['pop_size'],
                                      params['mutation_prob'],
                                      params['max_attempts'],
                                      params['max_iterations']
                                     )
    elif params['alg'] == "mimic":
        bf,fc,time_diff = run_mimic(problem,
                                    params['pop_size'],
                                    params['keep_pct'],
                                    params['max_attempts'],
                                    params['max_iterations']
                                     )
    return {'loss': -bf,'status': STATUS_OK, 'fitness_curve': fc, 'params': params, 'time':time_diff}

In [4]:
def continuous_peaks(params):
    fitness = mlrose.ContinuousPeaks(t_pct = params['t_pct'])
    problem = mlrose.DiscreteOpt(length = params['prob_size'], fitness_fn = fitness, maximize = True, max_val = 2)
    ints = np.random.randint(2, size = params['prob_size']) 
    if params['alg'] == "annealing":
        bf,fc,time_diff= run_annealing(problem,
                                       params['decay_ob'],
                                       params['init_temp'],
                                       ints,
                                       params['max_attempts'],
                                       params['max_iterations']
                                      )
    elif params['alg'] == "rhc":
        bf,fc,time_diff= run_rhc(problem,
                                 params['restart'],
                                 ints,
                                 params['max_attempts'],
                                 params['max_iterations']
                                )
        
    elif params['alg'] == "genetic":
        bf,fc,time_diff = run_genetic(problem,
                                      params['pop_size'],
                                      params['mutation_prob'],
                                      params['max_attempts'],
                                      params['max_iterations']
                                     )
    elif params['alg'] == "mimic":
        bf,fc,time_diff = run_mimic(problem,
                                      params['pop_size'],
                                      params['keep_pct'],
                                      params['max_attempts'],
                                      params['max_iterations']
                                     )
    return {'loss': -bf,'status': STATUS_OK, 'fitness_curve': fc, 'params': params, 'time':time_diff}

In [5]:
def n_queens(params):
    fitness = mlrose.Queens()
    problem = mlrose.DiscreteOpt(length = params['prob_size'], 
                                 fitness_fn = fitness, 
                                 maximize = False, 
                                 max_val = params['prob_size'])
    ints = range(0,params['prob_size']) 
    if params['alg'] == "annealing":
        bf,fc,time_diff= run_annealing(problem,
                                       params['decay_ob'],
                                       params['init_temp'],
                                       ints,
                                       params['max_attempts'],
                                       params['max_iterations']
                                      )
    elif params['alg'] == "rhc":
        bf,fc,time_diff= run_rhc(problem,
                                 params['restart'],
                                 ints,
                                 params['max_attempts'],
                                 params['max_iterations']
                                )

    elif params['alg'] == "genetic":
        bf,fc,time_diff = run_genetic(problem,
                                      params['pop_size'],
                                      params['mutation_prob'],
                                      params['max_attempts'],
                                      params['max_iterations']
                                     )
    elif params['alg'] == "mimic":
        bf,fc,time_diff = run_mimic(problem,
                                      params['pop_size'],
                                      params['keep_pct'],
                                      params['max_attempts'],
                                      params['max_iterations']
                                     )
    return {'loss': bf,'status': STATUS_OK, 'fitness_curve': fc, 'params': params, 'time':time_diff}

In [6]:
def ann_cp(max_ev):
    space = {'alg': hp.choice('alg',["annealing"]),
             't_pct': hp.choice('t_pct', np.logspace(-4,0,6)),
             'prob_size': hp.choice('prob_size', list(range(5,125,20))),
             'decay_ob': hp.choice('decay_ob',[("geom",0.25),
                                               ("geom",0.5),
                                               ("geom",0.7),
                                               ("geom",0.95),
                                               ("exp",1.58489319e-01),
                                               ("exp",2.51188643e+00),
                                               ("exp",3.98107171e+01)]),
             'init_temp': hp.choice('init_temp',np.arange(1.0,12.0,1.67)),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000, 100000])
    }
    trials = Trials()
    best = fmin(continuous_peaks, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("continuous_peak-annealing.p", "wb"))

def gen_cp(max_ev):

    space = {'alg': hp.choice('alg',["genetic"]),
             't_pct': hp.choice('t_pct', np.logspace(-4,0,6)),
             'prob_size': hp.choice('prob_size', list(range(5,125,20))),
             'pop_size': hp.choice('pop_size',[50,100,200,400]),
             'mutation_prob': hp.choice('mutation_prob',[0.001,0.01,0.1,0.2,0.5,0.8,1]),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000, 100000])
    }
    trials = Trials()
    best = fmin(continuous_peaks, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("continuous_peak-genetic.p", "wb"))

def mimic_cp(max_ev):
    space = {'alg': hp.choice('alg',["mimic"]),
             't_pct': hp.choice('t_pct', np.logspace(-4,0,6)),
             'prob_size': hp.choice('prob_size', list(range(5,125,20))),
             'pop_size': hp.choice('pop_size',[40,60,80,100,120,140]),
             'keep_pct': hp.choice('keep_pct',[0.2,0.5,0.7,0.9]),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000, 100000])
    }
    trials = Trials()
    best = fmin(continuous_peaks, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("continuous_peak-mimic.p", "wb"))
    

def rhc_cp(max_ev):
    space = {'alg': hp.choice('alg',["rhc"]),
             't_pct': hp.choice('t_pct', np.logspace(-4,0,6)),
             'prob_size': hp.choice('prob_size', list(range(5,125,20))),
             'restart': hp.choice('restart',list(range(0,15,1))),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000, 100000])
    }
    trials = Trials()
    best = fmin(continuous_peaks, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("continuous_peak-rhc.p", "wb"))

def ann_qn(max_ev):
    space = {'alg': hp.choice('alg',["annealing"]),
             'prob_size': hp.choice('prob_size', list(range(5,125,20))),
             'decay_ob': hp.choice('decay_ob',[("geom",0.25),
                                               ("geom",0.5),
                                               ("geom",0.7),
                                               ("geom",0.95),
                                               ("exp",1.58489319e-01),
                                               ("exp",2.51188643e+00),
                                               ("exp",3.98107171e+01)]),
             'init_temp': hp.choice('init_temp',np.arange(1.0,12.0,1.67)),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000, 100000])
    }
    trials = Trials()
    best = fmin(n_queens, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("queens-annealing.p", "wb"))

def gen_qn(max_ev):

    space = {'alg': hp.choice('alg',["genetic"]),
             'prob_size': hp.choice('prob_size', list(range(5,125,20))),
             'pop_size': hp.choice('pop_size',[50,100,200,400]),
             'mutation_prob': hp.choice('mutation_prob',[0.001,0.01,0.1,0.2,0.5,0.8,1]),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000, 100000])
    }
    trials = Trials()
    best = fmin(n_queens, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("queens-genetic.p", "wb"))

def mimic_qn(max_ev):
    space = {'alg': hp.choice('alg',["mimic"]),
             'prob_size': hp.choice('prob_size', list(range(5,125,20))),
             'pop_size': hp.choice('pop_size',[40,60,80,100,120,140]),
             'keep_pct': hp.choice('keep_pct',[0.2,0.5,0.7,0.9]),
             'max_attempts': hp.choice('max_attempts', [10, 100]),
             'max_iterations': hp.choice('max_iterations', [10, 100])
    }
    trials = Trials()
    best = fmin(n_queens, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("queens-mimic.p", "wb"))
    

def rhc_qn(max_ev):
    space = {'alg': hp.choice('alg',["rhc"]),
             'prob_size': hp.choice('prob_size', list(range(5,125,20))),
             'restart': hp.choice('restart',list(range(0,15,1))),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000, 100000])
    }
    trials = Trials()
    best = fmin(n_queens, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("queens-rhc.p", "wb"))
    
def ann_ff(max_ev):
    space = {'alg': hp.choice('alg',["annealing"]),
             'prob_size': hp.choice('prob_size', list(range(5,125,20))),
             'decay_ob': hp.choice('decay_ob',[("geom",0.25),
                                               ("geom",0.5),
                                               ("geom",0.7),
                                               ("geom",0.95),
                                               ("exp",1.58489319e-01),
                                               ("exp",2.51188643e+00),
                                               ("exp",3.98107171e+01)]),
             'init_temp': hp.choice('init_temp',np.arange(1.0,12.0,1.67)),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000, 100000])
    }
    trials = Trials()
    best = fmin(flipFlop, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("flipflop-annealing.p", "wb"))

def gen_ff(max_ev):

    space = {'alg': hp.choice('alg',["genetic"]),
             'prob_size': hp.choice('prob_size', list(range(5,125,20))),
             'pop_size': hp.choice('pop_size',[50,100,200,400]),
             'mutation_prob': hp.choice('mutation_prob',[0.001,0.01,0.1,0.2,0.5,0.8,1]),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000, 100000])
    }
    trials = Trials()
    best = fmin(flipFlop, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("flipflop-genetic.p", "wb"))

def mimic_ff(max_ev):
    space = {'alg': hp.choice('alg',["mimic"]),
             'prob_size': hp.choice('prob_size', list(range(5,125,20))),
             'pop_size': hp.choice('pop_size',[40,60,80,100,120,140]),
             'keep_pct': hp.choice('keep_pct',[0.2,0.5,0.7,0.9]),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000, 100000])
    }
    trials = Trials()
    best = fmin(flipFlop, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("flipflop-mimic.p", "wb"))
    

def rhc_ff(max_ev):
    space = {'alg': hp.choice('alg',["rhc"]),
             'prob_size': hp.choice('prob_size', list(range(5,125,20))),
             'restart': hp.choice('restart',list(range(0,5,100))),
             'max_attempts': hp.choice('max_attempts', [10, 100, 1000, 10000]),
             'max_iterations': hp.choice('max_iterations', [10, 100, 1000, 10000, 100000])
    }
    trials = Trials()
    best = fmin(flipFlop, space, algo=tpe.suggest, trials=trials, max_evals=max_ev)
    pickle.dump(trials, open("flipflop-rhc.p", "wb"))

In [7]:
EVAL = 300

#ann_cp(EVAL)
#gen_cp(100)
#mimic_cp(EVAL)
#rhc_cp(EVAL)

#ann_ff(EVAL)
#gen_ff(100)
#mimic_ff(EVAL)
#rhc_ff(EVAL)

#ann_qn(EVAL)
#gen_qn(100)
mimic_qn(15)
#rhc_qn(EVAL)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [1:00:39<00:00, 242.64s/trial, best loss: 0.0]


In [8]:
test = {}

test['model'] = nn_model

NameError: name 'nn_model' is not defined